In [4]:
# Road Trip Planner

from math import *

# FILENAME = 'usa_cities.csv'
FILENAME = 'california.csv'
START = 'San Francisco, California'
# END = 'New York, New York'
END = 'San Diego, California'
KM_PER_DAY = 75

def haversine_formula(lat_i, lon_i, lat_j, lon_j):    
    # Haversine formula for distances between points on a sphere
    # https://en.wikipedia.org/wiki/Haversine_formula
    lat_i = lat_i * 3.14 / 180
    lon_i = lon_i * 3.14 / 180
    lat_j = lat_j * 3.14 / 180
    lon_j = lon_j * 3.14 / 180
    dlat = lat_j - lat_i
    dlon = lon_j - lon_i
    a = (sin(dlat/2)*sin(dlat/2)) + cos(lat_i) * cos(lat_j) * (sin(dlon/2)*sin(dlon/2))
    c = 2 * atan2( a**0.5, (1-a)**0.5 )
    d = 6373  * c  # R is 'a' radius of earth
    return d

# Read data file
input = open(FILENAME, encoding='utf-8').readlines()

# column headers
cities = []
lat = []
lon = []
counties = []
states = []

for l in range(0, len(input)):
    line = input[l]

    # Divide in columns
    columns = line.replace('"', '').split(',')

    # ignore header and empty lines
    if len(columns) > 0 and columns[0][0] != '#':
        cities.append(columns[0])
        counties.append(columns[1])
        states.append(columns[2])
        lat.append(eval(columns[3]))
        lon.append(eval(columns[4]))

print('Read', str(len(cities)), 'cities')

# Find query cities in database
start_city = START.split(',')[0].strip()
start_state = START.split(',')[1].strip()

end_city = END.split(',')[0].strip()
end_state = END.split(',')[1].strip()

start_idx = None
end_idx = None

for i in range(0, len(cities)):
  if cities[i] == start_city and states[i] == start_state:
    print('Found start', i)
    start_idx = i
  elif cities[i] == end_city and states[i] == end_state:
    print('Found end', i)
    end_idx = i

# Do search
if start_idx != None and end_idx != None:

    # Find 'best' route
    route = [(start_city, start_state)]
    
    current_city = (start_city, start_state)
    dead_end = False
    do_search = True
    while do_search:
        city = current_city[0]
        state = current_city[1]
        
        # Get city index in lists
        for i in range(0, len(cities)):
            if cities[i] == city and states[i] == state:
                cur_idx = i

        # Get neighbors
        neighbors = []
        for j in range(0, len(cities)):
            
            d = haversine_formula(lat[cur_idx], lon[cur_idx], lat[j], lon[j])
            
            if d <= KM_PER_DAY:
                neighbors.append((j, cities[j], states[j]))
        
        if (end_idx, end_city, end_state) in neighbors:  # end search
            pick = (end_city, end_state)
            do_search = False
        else:
            min_d = 99999999999999999999
            pick = None
            
            # Calculate closest neighbor to destination
            for i in range(0, len(neighbors)):
                nidx = neighbors[i][0]
                nname = neighbors[i][1]
                nstate = neighbors[i][2]

                d = haversine_formula(lat[nidx], lon[nidx], lat[end_idx], lon[end_idx])

                if d <= min_d and (nname, nstate) not in route:
                    min_d = d
                    pick = (nname, nstate)

            if pick == None:  # did not find a suitable neighbor
              do_search = False
              dead_end = True
                
        route.append(pick)
        current_city = pick

    # Print result
    if dead_end: print('Cannot find route with requested parameters')
    else:
        print('Proposed Route:')
        counter = 1
        for step in route:
          city = step[0]
          state = step[1]
          print('Stop #' + str(counter), ':', city + ',', state)
          counter += 1
else: print('Could not find requested cities')


Read 1242 cities
Found end 957
Found start 960
Found start 961
Proposed Route:
Stop #1 : San Francisco, California
Stop #2 : Coyote, California
Stop #3 : Chualar, California
Stop #4 : Jolon, California
Stop #5 : Atascadero, California
Stop #6 : Casmalia, California
Stop #7 : Goleta, California
Stop #8 : Ventura, California
Stop #9 : Pacific Palisades, California
Stop #10 : Costa Mesa, California
Stop #11 : Oceanside, California
Stop #12 : San Diego, California
